# Overview

This scraper loads the TMDB data file with IMDB IDs, containing 10,000 movie entries, and scrapes the Box Office Mojo website for any/all of the following:
    * budget
    * domestic gross
    * worldwide gross
    * studio
    * MPAA rating

# Library Imports

In [4]:
import pandas as pd
import requests
import re
import numpy as np
from bs4 import BeautifulSoup as bs

# Scraping Functions

In [145]:
# function that converts our string to a search string
def format_title(title):
    text = str(title)
    result = re.sub(r"[,@\'?\.$%_:â()-]", "", text, flags=re.I)
    result = re.sub(r"\s+","+", result, flags = re.I)
    return result

In [146]:
# function to convert the raw numbers into integers
def convert_number(x):
    '''Takes in a string formatted number that starts with $ and may include commas, and returns that 
    number as a whole integer that can be used in calculations'''
    try:
        x = x[1:]
        x = x.replace(',', '')
        x = int(x)
        return x
    except:
        print('No number to convert')
        return None

In [147]:
# function that receives the followurl appendation for the IMDB id and follows it
def follow_imdb(movie_id):
    followurl = 'https://www.boxofficemojo.com/title/' + movie_id
    page = requests.get(followurl)
    soup = bs(page.content, 'html.parser')
    return soup

In [148]:
# Function that returns the domestic and foreign gross
def gross(movie_page):
    try:
        search1 = movie_page.find('div', class_='a-section a-spacing-none mojo-performance-summary-table')
        dgross = search1.find_all('span', class_='money')[0].text
        fgross = search1.find_all('span', class_='money')[1].text
        dgross = convert_number(dgross)
        fgross = convert_number(fgross)
        return dgross, fgross
    except:
        return None, None

In [149]:
# function that takes in the product page and gets the studio name
def studio_name(soup):
    try:
        search1 = soup.find('div', class_='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile')
        search2 = search1.find_all('div', class_='a-section a-spacing-none')[0].find_all("span")
        if search2[0].text == 'Domestic Distributor':
            studio = search2[1].text.replace('See full company information', '').rstrip()
            return studio
        else:
            return None
    except:
        print('error in studio function')
        return None

In [150]:
# function that takes in the product page and gets the budget
def find_budget(soup):
    try:
        search1 = soup.find('div', class_='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile')
        search2 = search1.find_all('div', class_='a-section a-spacing-none')[2].find_all("span")
        if search2[0].text == 'Budget':
            budget = search2[1].text
            budget = convert_number(budget)
            return budget
        else:
            return None
    except:
        print('error in budget function')
        return None

In [151]:
# function that takes in the product page and gives the MPAA rating
def find_mpaa(soup):
    try:
        search1 = soup.find('div', class_='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile')
        div_search = search1.find_all('div', class_='a-section a-spacing-none')
        entries = len(div_search)
        
        for x in range(0, entries):
            search = div_search[x].find_all("span")
            if search[0].text == 'MPAA':
                print('checking 1')
                rating = search[1].text
                return rating
            else: continue
    except:
        print('error in rating function')
        return None

In [152]:
# Master Scraping Function

def webscraper(scrapingset):
    count = 1
    length = scrapingset.shape[0]
    for ind in scrapingset.index:   
        movie_id = str(scrapingset['id'][ind])
        print(('Item {} / {} - {}').format(count, length, movie_id))
        movie_page = follow_imdb(movie_id)
        count += 1

        rating = find_mpaa(movie_page)
        print(rating)
        scrapingset['rating'][ind] = rating
        
        studio = studio_name(movie_page)
        scrapingset['studio'][ind] = studio
   
        dgross, fgross = gross(movie_page)
        scrapingset['dom_gross'][ind] = dgross
        scrapingset['for_gross'][ind] = fgross
    
        budget = find_budget(movie_page)
        scrapingset['budget'][ind] = budget
    
    return scrapingset

# TO DO BEFORE REDOING

Round the numbers so they don't have decimal places

Add a scraper for rating!

# Data Import

In [153]:
# import tmdb_imdb files
df = pd.read_csv('api_data/tmdb_imdb_ids.csv')

In [154]:
df

,id,popularity,vote_count,genre_ids,title,vote_average,release_date
0,tt2380307,520.621,12639,"[16, 10751, 35, 12, 14, 10402]",Coco,8.2,2017-10-27
1,tt7286456,330.357,15378,"[80, 53, 18]",Joker,8.2,2019-10-02
2,tt7349950,288.149,5133,"[27, 14]",It Chapter Two,6.9,2019-09-04
3,tt4520988,257.243,6344,"[16, 10751, 12, 35, 14]",Frozen II,7.3,2019-11-20
4,tt7975244,216.184,4925,"[12, 35, 14]",Jumanji The Next Level,7.0,2019-12-04
...,...,...,...,...,...,...,...
9995,tt0349080,4.837,61,"[35, 18, 10749]",All About Anna,3.2,2005-11-24
9996,tt0366178,4.836,67,"[16, 878]",Program,6.9,2003-02-07
9997,tt2501618,4.830,72,"[27, 14, 16]",Dont Hug Me Im Scared,7.5,2011-07-25
9998,tt0340376,4.830,40,"[18, 53]",The Statement,5.7,2003-12-12


In [135]:
#We need nothing but the id from this dataframe
df.drop(columns=['popularity', 'vote_count', 'genre_ids', 'title', 'vote_average', 'release_date'], inplace=True)

In [136]:
# We're prepping our dataframe structure for the information that we plan to scrape
df['studio'] = ''
df['rating'] = ''
df['budget'] = ''
df['dom_gross'] = ''
df['for_gross'] = ''
df

,id,studio,rating,budget,dom_gross,for_gross
0,tt2380307,,,,,
1,tt7286456,,,,,
2,tt7349950,,,,,
3,tt4520988,,,,,
4,tt7975244,,,,,
...,...,...,...,...,...,...
9995,tt0349080,,,,,
9996,tt0366178,,,,,
9997,tt2501618,,,,,
9998,tt0340376,,,,,


# Testing Set

Before we scrape for 10k returns, we will do a small test scrape

In [137]:
test1 = pd.DataFrame(columns = ['id', 'rating', 'studio', 'budget', 'dom_gross', 'for_gross'])

In [138]:
ids = ['tt1825683', 'tt0349080', 'tt7286456']
test1['id'] = np.array(ids)

In [139]:
test1

,id,rating,studio,budget,dom_gross,for_gross
0,tt1825683,NaN,NaN,NaN,NaN,NaN
1,tt0349080,NaN,NaN,NaN,NaN,NaN
2,tt7286456,NaN,NaN,NaN,NaN,NaN


In [140]:
test1 = webscraper(test1)

Item 1 / 3 - tt1825683
checking 1
PG-13
Item 2 / 3 - tt0349080
None
Item 3 / 3 - tt7286456
checking 1
R


In [141]:
test1

,id,rating,studio,budget,dom_gross,for_gross
0,tt1825683,PG-13,Walt Disney Studios Motion Pictures,None,700426566,647035818
1,tt0349080,None,None,None,None,None
2,tt7286456,R,Warner Bros.,55000000,335451311,738800000


In [142]:
#A bigger test set
test2 = df[0:10]
test2

,id,studio,rating,budget,dom_gross,for_gross
0,tt2380307,,,,,
1,tt7286456,,,,,
2,tt7349950,,,,,
3,tt4520988,,,,,
4,tt7975244,,,,,
5,tt4154756,,,,,
6,tt5168832,,,,,
7,tt9465600,,,,,
8,tt0414982,,,,,
9,tt6806448,,,,,


In [143]:
test2 = webscraper(test2)

Item 1 / 10 - tt2380307
checking 1
PG
Item 2 / 10 - tt7286456


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


checking 1
R
Item 3 / 10 - tt7349950
checking 1
R
Item 4 / 10 - tt4520988
checking 1
PG
Item 5 / 10 - tt7975244
checking 1
PG-13
Item 6 / 10 - tt4154756
checking 1
PG-13
Item 7 / 10 - tt5168832
None
Item 8 / 10 - tt9465600
checking 1
R
Item 9 / 10 - tt0414982
checking 1
R
Item 10 / 10 - tt6806448
None


In [144]:
test2

,id,studio,rating,budget,dom_gross,for_gross
0,tt2380307,Walt Disney Studios Motion Pictures,PG,None,210460015,597357873
1,tt7286456,Warner Bros.,R,55000000,335451311,738800000
2,tt7349950,Warner Bros.,R,79000000,211593228,261500000
3,tt4520988,Walt Disney Studios Motion Pictures,PG,150000000,477373578,972653355
4,tt7975244,Sony Pictures Entertainment (SPE),PG-13,125000000,320314960,479744747
5,tt4154756,Walt Disney Studios Motion Pictures,PG-13,None,678815482,1369544272
6,tt5168832,None,None,None,None,None
7,tt9465600,None,R,None,None,None
8,tt0414982,New Line Cinema,R,25000000,54098051,64792221
9,tt6806448,Universal Pictures,None,200000000,173956935,585100000


# The Big Scrape

We're ready to do the big scrape!
We'll break our frame of 10,000 entries into 5 smaller ones in case of any errors.

In [26]:
df1 = df[0:2000]

In [27]:
df1 = webscraper(df1)

Item 1 / 2001 - tt2380307


C:\Users\thren\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 2001 - tt7286456
Item 3 / 2001 - tt7349950
Item 4 / 2001 - tt4520988
Item 5 / 2001 - tt7975244
Item 6 / 2001 - tt4154756
Item 7 / 2001 - tt5168832
Item 8 / 2001 - tt9465600
Item 9 / 2001 - tt0414982
Item 10 / 2001 - tt6806448
Item 11 / 2001 - tt4126476
Item 12 / 2001 - tt6450804
Item 13 / 2001 - tt6105098
Item 14 / 2001 - tt4154796
Item 15 / 2001 - tt4777008
Item 16 / 2001 - tt4094724
Item 17 / 2001 - tt6133466
Item 18 / 2001 - tt8579674
Item 19 / 2001 - tt6320628
Item 20 / 2001 - tt1935859
Item 21 / 2001 - tt1979376
Item 22 / 2001 - tt1935896
Item 23 / 2001 - tt6146586
Item 24 / 2001 - tt2527338
Item 25 / 2001 - tt4281724
Item 26 / 2001 - tt1025100
Item 27 / 2001 - tt2709692
Item 28 / 2001 - tt4633694
Item 29 / 2001 - tt0329774
Item 30 / 2001 - tt1560220
Item 31 / 2001 - tt8106534
Item 32 / 2001 - tt7287680
Item 33 / 2001 - tt2184339
Item 34 / 2001 - tt1620981
Item 35 / 2001 - tt6095472
Item 36 / 2001 - tt5884052
Item 37 / 2001 - tt5220122
Item 38 / 2001 - tt6139732
Item 39 /

Item 299 / 2001 - tt6643972
Item 300 / 2001 - tt9382596
Item 301 / 2001 - tt7831358
Item 302 / 2001 - tt8186318
Item 303 / 2001 - tt4007502
Item 304 / 2001 - tt1790809
Item 305 / 2001 - tt6645614
Item 306 / 2001 - tt1267297
Item 307 / 2001 - tt0434409
Item 308 / 2001 - tt8752440
Item 309 / 2001 - tt1979388
Item 310 / 2001 - tt3584496
Item 311 / 2001 - tt2582802
Item 312 / 2001 - tt4790268
Item 313 / 2001 - tt0385887
Item 314 / 2001 - tt1413492
Item 315 / 2001 - tt4158096
Item 316 / 2001 - tt0816711
Item 317 / 2001 - tt1411697
Item 318 / 2001 - tt1029360
Item 319 / 2001 - tt1261978
Item 320 / 2001 - tt2911666
Item 321 / 2001 - tt0372784
Item 322 / 2001 - tt0167261
Item 323 / 2001 - tt10767720
Item 324 / 2001 - tt0936501
Item 325 / 2001 - tt3235888
Item 326 / 2001 - tt3846674
Item 327 / 2001 - tt1981115
Item 328 / 2001 - tt1951261
Item 329 / 2001 - tt0252866
Item 330 / 2001 - tt6182908
Item 331 / 2001 - tt0472043
Item 332 / 2001 - tt0449088
Item 333 / 2001 - tt3605418
Item 334 / 2001 - t

Item 592 / 2001 - tt0328828
Item 593 / 2001 - tt1563742
Item 594 / 2001 - tt0373883
Item 595 / 2001 - tt0328880
Item 596 / 2001 - tt1638002
Item 597 / 2001 - tt7339792
Item 598 / 2001 - tt2537176
Item 599 / 2001 - tt8031422
Item 600 / 2001 - tt0780521
Item 601 / 2001 - tt9071322
Item 602 / 2001 - tt4257926
Item 603 / 2001 - tt4950110
Item 604 / 2001 - tt1598778
Item 605 / 2001 - tt0489099
Item 606 / 2001 - tt6292852
Item 607 / 2001 - tt1853728
Item 608 / 2001 - tt5822648
Item 609 / 2001 - tt1179056
Item 610 / 2001 - tt4799066
Item 611 / 2001 - tt0498353
Item 612 / 2001 - tt9815714
Item 613 / 2001 - tt1618434
Item 614 / 2001 - tt1190080
Item 615 / 2001 - tt8235660
Item 616 / 2001 - tt2091256
Item 617 / 2001 - tt0266697
Item 618 / 2001 - tt0134084
Item 619 / 2001 - tt3440298
Item 620 / 2001 - tt6017942
Item 621 / 2001 - tt8179388
Item 622 / 2001 - tt0289043
Item 623 / 2001 - tt3774802
Item 624 / 2001 - tt0317705
Item 625 / 2001 - tt3195644
Item 626 / 2001 - tt2231253
Item 627 / 2001 - tt

Item 883 / 2001 - tt7401588
Item 884 / 2001 - tt2404435
Item 885 / 2001 - tt0451079
Item 886 / 2001 - tt1351685
Item 887 / 2001 - tt7052244
Item 888 / 2001 - tt1139668
Item 889 / 2001 - tt1700841
Item 890 / 2001 - tt5715410
Item 891 / 2001 - tt5360996
Item 892 / 2001 - tt0309593
Item 893 / 2001 - tt1485796
Item 894 / 2001 - tt6857112
Item 895 / 2001 - tt0884328
Item 896 / 2001 - tt0383206
Item 897 / 2001 - tt5737536
Item 898 / 2001 - tt1985966
Item 899 / 2001 - tt5208950
Item 900 / 2001 - tt4824308
Item 901 / 2001 - tt0265208
Item 902 / 2001 - tt0301470
Item 903 / 2001 - tt1217213
Item 904 / 2001 - tt1478839
Item 905 / 2001 - tt0265086
Item 906 / 2001 - tt0840361
Item 907 / 2001 - tt2096672
Item 908 / 2001 - tt4287320
Item 909 / 2001 - tt4976192
Item 910 / 2001 - tt1821694
Item 911 / 2001 - tt0389790
Item 912 / 2001 - tt1192628
Item 913 / 2001 - tt1583421
Item 914 / 2001 - tt6017756
Item 915 / 2001 - tt7040874
Item 916 / 2001 - tt3079380
Item 917 / 2001 - tt1285016
Item 918 / 2001 - tt

Item 1170 / 2001 - tt3402236
Item 1171 / 2001 - tt5898034
Item 1172 / 2001 - tt1000774
Item 1173 / 2001 - tt1041829
Item 1174 / 2001 - tt0315327
Item 1175 / 2001 - tt0408236
Item 1176 / 2001 - tt0760329
Item 1177 / 2001 - tt0443676
Item 1178 / 2001 - tt0419946
Item 1179 / 2001 - tt0146316
Item 1180 / 2001 - tt6185658
Item 1181 / 2001 - tt1245492
Item 1182 / 2001 - tt6679794
Item 1183 / 2001 - tt1216515
Item 1184 / 2001 - tt7448180
Item 1185 / 2001 - tt0461770
Item 1186 / 2001 - tt4663548
Item 1187 / 2001 - tt1472584
Item 1188 / 2001 - tt2004420
Item 1189 / 2001 - tt2113007
Item 1190 / 2001 - tt6535880
Item 1191 / 2001 - tt0388482
Item 1192 / 2001 - tt4536768
Item 1193 / 2001 - tt0267804
Item 1194 / 2001 - tt0306685
Item 1195 / 2001 - tt1023481
Item 1196 / 2001 - tt2717822
Item 1197 / 2001 - tt6802308
Item 1198 / 2001 - tt6269368
Item 1199 / 2001 - tt2702348
Item 1200 / 2001 - tt4229236
Item 1201 / 2001 - tt0960144
Item 1202 / 2001 - tt0362165
Item 1203 / 2001 - tt6628394
Item 1204 / 20

Item 1453 / 2001 - tt0472033
Item 1454 / 2001 - tt2788710
Item 1455 / 2001 - tt2002718
Item 1456 / 2001 - tt1392214
Item 1457 / 2001 - tt4555426
Item 1458 / 2001 - tt0465375
Item 1459 / 2001 - tt6987770
Item 1460 / 2001 - tt2548396
Item 1461 / 2001 - tt0865556
Item 1462 / 2001 - tt5834760
Item 1463 / 2001 - tt0402022
Item 1464 / 2001 - tt0431308
Item 1465 / 2001 - tt1185834
Item 1466 / 2001 - tt0388419
Item 1467 / 2001 - tt1265998
Item 1468 / 2001 - tt5702566
Item 1469 / 2001 - tt0299172
Item 1470 / 2001 - tt7312940
Item 1471 / 2001 - tt1194173
Item 1472 / 2001 - tt1893256
Item 1473 / 2001 - tt1905040
Item 1474 / 2001 - tt6722030
Item 1475 / 2001 - tt0286106
Item 1476 / 2001 - tt0246578
Item 1477 / 2001 - tt0365748
Item 1478 / 2001 - tt2261287
Item 1479 / 2001 - tt0120681
Item 1480 / 2001 - tt1517260
Item 1481 / 2001 - tt0844479
Item 1482 / 2001 - tt4695012
Item 1483 / 2001 - tt7533152
Item 1484 / 2001 - tt6108178
Item 1485 / 2001 - tt0805526
Item 1486 / 2001 - tt0433386
Item 1487 / 20

Item 1736 / 2001 - tt0989757
Item 1737 / 2001 - tt1524575
Item 1738 / 2001 - tt0389722
Item 1739 / 2001 - tt2404311
Item 1740 / 2001 - tt0181689
Item 1741 / 2001 - tt1450334
Item 1742 / 2001 - tt1270761
Item 1743 / 2001 - tt1921064
Item 1744 / 2001 - tt0479997
Item 1745 / 2001 - tt0798817
Item 1746 / 2001 - tt0437954
Item 1747 / 2001 - tt1509767
Item 1748 / 2001 - tt5783956
Item 1749 / 2001 - tt5431890
Item 1750 / 2001 - tt0322589
Item 1751 / 2001 - tt6476140
Item 1752 / 2001 - tt7946422
Item 1753 / 2001 - tt0477080
Item 1754 / 2001 - tt4779496
Item 1755 / 2001 - tt1034415
Item 1756 / 2001 - tt6127004
Item 1757 / 2001 - tt0465940
Item 1758 / 2001 - tt0385752
Item 1759 / 2001 - tt0368447
Item 1760 / 2001 - tt3361792
Item 1761 / 2001 - tt0259685
Item 1762 / 2001 - tt2726560
Item 1763 / 2001 - tt1568346
Item 1764 / 2001 - tt0429591
Item 1765 / 2001 - tt2024469
Item 1766 / 2001 - tt4356480
Item 1767 / 2001 - tt1478338
Item 1768 / 2001 - tt4731136
Item 1769 / 2001 - tt5776858
Item 1770 / 20

In [36]:
df2 = df[2000:4000]

In [38]:
df2 = webscraper(df2)

Item 1 / 1999 - tt0481369


C:\Users\thren\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1999 - tt3787590
Item 3 / 1999 - tt0274546
Item 4 / 1999 - tt5093026
Item 5 / 1999 - tt0288045
Item 6 / 1999 - tt1430612
Item 7 / 1999 - tt1814614
Item 8 / 1999 - tt0443473
Item 9 / 1999 - tt0407121
Item 10 / 1999 - tt6217804
Item 11 / 1999 - tt1596365
Item 12 / 1999 - tt2476918
Item 13 / 1999 - tt7549996
Item 14 / 1999 - tt1579247
Item 15 / 1999 - tt0960890
Item 16 / 1999 - tt0381849
Item 17 / 1999 - tt3451230
Item 18 / 1999 - tt2180411
Item 19 / 1999 - tt7319822
Item 20 / 1999 - tt9381998
Item 21 / 1999 - tt3467412
Item 22 / 1999 - tt6946580
Item 23 / 1999 - tt0970452
Item 24 / 1999 - tt0443701
Item 25 / 1999 - tt0485947
Item 26 / 1999 - tt4677938
Item 27 / 1999 - tt4943998
Item 28 / 1999 - tt7983844
Item 29 / 1999 - tt6048930
Item 30 / 1999 - tt1564585
Item 31 / 1999 - tt8457394
Item 32 / 1999 - tt2752758
Item 33 / 1999 - tt1560985
Item 34 / 1999 - tt0303933
Item 35 / 1999 - tt0806203
Item 36 / 1999 - tt1205535
Item 37 / 1999 - tt1288403
Item 38 / 1999 - tt1212450
Item 39 /

Item 298 / 1999 - tt2100546
Item 299 / 1999 - tt0964517
Item 300 / 1999 - tt0442933
Item 301 / 1999 - tt1599351
Item 302 / 1999 - tt1002563
Item 303 / 1999 - tt2172934
Item 304 / 1999 - tt0491203
Item 305 / 1999 - tt1447972
Item 306 / 1999 - tt2205401
Item 307 / 1999 - tt0283426
Item 308 / 1999 - tt2039338
Item 309 / 1999 - tt5698320
Item 310 / 1999 - tt0334541
Item 311 / 1999 - tt3164256
Item 312 / 1999 - tt0317198
Item 313 / 1999 - tt1596363
Item 314 / 1999 - tt0206275
Item 315 / 1999 - tt0457419
Item 316 / 1999 - tt2349460
Item 317 / 1999 - tt0280590
Item 318 / 1999 - tt8722440
Item 319 / 1999 - tt7985692
Item 320 / 1999 - tt6182078
Item 321 / 1999 - tt1925431
Item 322 / 1999 - tt0405469
Item 323 / 1999 - tt1971325
Item 324 / 1999 - tt6019206
Item 325 / 1999 - tt5502946
Item 326 / 1999 - tt1205489
Item 327 / 1999 - tt5513260
Item 328 / 1999 - tt1477715
Item 329 / 1999 - tt8510488
Item 330 / 1999 - tt1535109
Item 331 / 1999 - tt1251757
Item 332 / 1999 - tt0756683
Item 333 / 1999 - tt

Item 590 / 1999 - tt0469111
Item 591 / 1999 - tt0758758
Item 592 / 1999 - tt5563330
Item 593 / 1999 - tt1179034
Item 594 / 1999 - tt2870756
Item 595 / 1999 - tt2582784
Item 596 / 1999 - tt2513074
Item 597 / 1999 - tt0865957
Item 598 / 1999 - tt0844760
Item 599 / 1999 - tt3598222
Item 600 / 1999 - tt0454848
Item 601 / 1999 - tt0486761
Item 602 / 1999 - tt2315596
Item 603 / 1999 - tt3289724
Item 604 / 1999 - tt1196339
Item 605 / 1999 - tt1212419
Item 606 / 1999 - tt4682786
Item 607 / 1999 - tt4215766
Item 608 / 1999 - tt1259998
Item 609 / 1999 - tt0976051
Item 610 / 1999 - tt0245562
Item 611 / 1999 - tt0457275
Item 612 / 1999 - tt5044656
Item 613 / 1999 - tt0221027
Item 614 / 1999 - tt6997574
Item 615 / 1999 - tt4797160
Item 616 / 1999 - tt0268397
Item 617 / 1999 - tt0889583
Item 618 / 1999 - tt8485364
Item 619 / 1999 - tt1024255
Item 620 / 1999 - tt1131724
Item 621 / 1999 - tt5390066
Item 622 / 1999 - tt2396421
Item 623 / 1999 - tt0866439
Item 624 / 1999 - tt7608418
Item 625 / 1999 - tt

Item 883 / 1999 - tt0360139
Item 884 / 1999 - tt0465624
Item 885 / 1999 - tt0239641
Item 886 / 1999 - tt0120913
Item 887 / 1999 - tt0815236
Item 888 / 1999 - tt0326769
Item 889 / 1999 - tt4009278
Item 890 / 1999 - tt2172935
Item 891 / 1999 - tt7358936
Item 892 / 1999 - tt0274166
Item 893 / 1999 - tt2282016
Item 894 / 1999 - tt0238546
Item 895 / 1999 - tt5851786
Item 896 / 1999 - tt3352390
Item 897 / 1999 - tt0881320
Item 898 / 1999 - tt1186373
Item 899 / 1999 - tt0278488
Item 900 / 1999 - tt2279339
Item 901 / 1999 - tt7689966
Item 902 / 1999 - tt2582502
Item 903 / 1999 - tt0190590
Item 904 / 1999 - tt0183649
Item 905 / 1999 - tt0413895
Item 906 / 1999 - tt0995868
Item 907 / 1999 - tt0263757
Item 908 / 1999 - tt1600058
Item 909 / 1999 - tt1282052
Item 910 / 1999 - tt0356618
Item 911 / 1999 - tt1135084
Item 912 / 1999 - tt1120985
Item 913 / 1999 - tt1886644
Item 914 / 1999 - tt2400685
Item 915 / 1999 - tt0387808
Item 916 / 1999 - tt3733774
Item 917 / 1999 - tt1580426
Item 918 / 1999 - tt

Item 1170 / 1999 - tt0993842
Item 1171 / 1999 - tt0221218
Item 1172 / 1999 - tt6288124
Item 1173 / 1999 - tt1449283
Item 1174 / 1999 - tt7883022
Item 1175 / 1999 - tt0255605
Item 1176 / 1999 - tt4463816
Item 1177 / 1999 - tt1591479
Item 1178 / 1999 - tt0426592
Item 1179 / 1999 - tt1243974
Item 1180 / 1999 - tt1504320
Item 1181 / 1999 - tt3111516
Item 1182 / 1999 - tt5513770
Item 1183 / 1999 - tt1435513
Item 1184 / 1999 - tt0293508
Item 1185 / 1999 - tt0253658
Item 1186 / 1999 - tt0295701
Item 1187 / 1999 - tt0471042
Item 1188 / 1999 - tt5057140
Item 1189 / 1999 - tt6678950
Item 1190 / 1999 - tt0480687
Item 1191 / 1999 - tt4538916
Item 1192 / 1999 - tt0376105
Item 1193 / 1999 - tt9028880
Item 1194 / 1999 - tt0857376
Item 1195 / 1999 - tt0418689
Item 1196 / 1999 - tt4437640
Item 1197 / 1999 - tt0405422
Item 1198 / 1999 - tt2372251
Item 1199 / 1999 - tt0870111
Item 1200 / 1999 - tt4595882
Item 1201 / 1999 - tt0381971
Item 1202 / 1999 - tt1222817
Item 1203 / 1999 - tt0281373
Item 1204 / 19

Item 1453 / 1999 - tt1058017
Item 1454 / 1999 - tt3068194
Item 1455 / 1999 - tt2900624
Item 1456 / 1999 - tt2117905
Item 1457 / 1999 - tt1850374
Item 1458 / 1999 - tt1356864
Item 1459 / 1999 - tt0300051
Item 1460 / 1999 - tt1175709
Item 1461 / 1999 - tt6952960
Item 1462 / 1999 - tt6217608
Item 1463 / 1999 - tt3173594
Item 1464 / 1999 - tt1230204
Item 1465 / 1999 - tt1491044
Item 1466 / 1999 - tt1287845
Item 1467 / 1999 - tt4058836
Item 1468 / 1999 - tt2080374
Item 1469 / 1999 - tt3603808
Item 1470 / 1999 - tt9036126
Item 1471 / 1999 - tt0393109
Item 1472 / 1999 - tt0484740
Item 1473 / 1999 - tt3399484
Item 1474 / 1999 - tt2246673
Item 1475 / 1999 - tt5475710
Item 1476 / 1999 - tt2357129
Item 1477 / 1999 - tt0260866
Item 1478 / 1999 - tt2980592
Item 1479 / 1999 - tt2417712
Item 1480 / 1999 - tt0228333
Item 1481 / 1999 - tt1306980
Item 1482 / 1999 - tt1407061
Item 1483 / 1999 - tt0376479
Item 1484 / 1999 - tt0337711
Item 1485 / 1999 - tt0257568
Item 1486 / 1999 - tt5340882
Item 1487 / 19

Item 1736 / 1999 - tt0808279
Item 1737 / 1999 - tt4291600
Item 1738 / 1999 - tt3174376
Item 1739 / 1999 - tt0365686
Item 1740 / 1999 - tt0265666
Item 1741 / 1999 - tt3760922
Item 1742 / 1999 - tt1792647
Item 1743 / 1999 - tt3297330
Item 1744 / 1999 - tt4733640
Item 1745 / 1999 - tt3286560
Item 1746 / 1999 - tt0209958
Item 1747 / 1999 - tt1315981
Item 1748 / 1999 - tt1300853
Item 1749 / 1999 - tt0489049
Item 1750 / 1999 - tt3630556
Item 1751 / 1999 - tt0280460
Item 1752 / 1999 - tt6083648
Item 1753 / 1999 - tt1229340
Item 1754 / 1999 - tt0166276
Item 1755 / 1999 - tt0810922
Item 1756 / 1999 - tt5481404
Item 1757 / 1999 - tt1541160
Item 1758 / 1999 - tt5224012
Item 1759 / 1999 - tt2545440
Item 1760 / 1999 - tt2980648
Item 1761 / 1999 - tt0186589
Item 1762 / 1999 - tt1234541
Item 1763 / 1999 - tt0270560
Item 1764 / 1999 - tt2515034
Item 1765 / 1999 - tt1662293
Item 1766 / 1999 - tt0480271
Item 1767 / 1999 - tt1083448
Item 1768 / 1999 - tt2383068
Item 1769 / 1999 - tt0433400
Item 1770 / 19

In [30]:
df3 = df[4000:6000]

In [43]:
df3 = webscraper(df3)

Item 1 / 2000 - tt1416801


C:\Users\thren\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 2000 - tt1391034
Item 3 / 2000 - tt3188560
Item 4 / 2000 - tt0395169
Item 5 / 2000 - tt0244970
Item 6 / 2000 - tt0264616
Item 7 / 2000 - tt0896534
Item 8 / 2000 - tt0219854
Item 9 / 2000 - tt1571243
Item 10 / 2000 - tt0325537
Item 11 / 2000 - tt0245120
Item 12 / 2000 - tt1007029
Item 13 / 2000 - tt1817081
Item 14 / 2000 - tt4758646
Item 15 / 2000 - tt0434215
Item 16 / 2000 - tt1114698
Item 17 / 2000 - tt0372237
Item 18 / 2000 - tt1833844
Item 19 / 2000 - tt0301526
Item 20 / 2000 - tt3488710
Item 21 / 2000 - tt1104835
Item 22 / 2000 - tt2101441
Item 23 / 2000 - tt4348012
Item 24 / 2000 - tt0279730
Item 25 / 2000 - tt4034354
Item 26 / 2000 - tt1790886
Item 27 / 2000 - tt0264472
Item 28 / 2000 - tt1313092
Item 29 / 2000 - tt2802136
Item 30 / 2000 - tt0443295
Item 31 / 2000 - tt1487931
Item 32 / 2000 - tt2145829
Item 33 / 2000 - tt2450186
Item 34 / 2000 - tt0838283
Item 35 / 2000 - tt7125860
Item 36 / 2000 - tt1104733
Item 37 / 2000 - tt0866437
Item 38 / 2000 - tt3672742
Item 39 /

Item 299 / 2000 - tt3438640
Item 300 / 2000 - tt1653665
Item 301 / 2000 - tt0368008
Item 302 / 2000 - tt0445922
Item 303 / 2000 - tt4129870
Item 304 / 2000 - tt0893382
Item 305 / 2000 - tt3864024
Item 306 / 2000 - tt0337697
Item 307 / 2000 - tt0430922
Item 308 / 2000 - tt0337909
Item 309 / 2000 - tt1412386
Item 310 / 2000 - tt1532503
Item 311 / 2000 - tt0940656
Item 312 / 2000 - tt0164212
Item 313 / 2000 - tt5460880
Item 314 / 2000 - tt8884364
Item 315 / 2000 - tt0211933
Item 316 / 2000 - tt0486578
Item 317 / 2000 - tt8488126
Item 318 / 2000 - tt6204340
Item 319 / 2000 - tt1535970
Item 320 / 2000 - tt7903530
Item 321 / 2000 - tt0303714
Item 322 / 2000 - tt1277737
Item 323 / 2000 - tt0376108
Item 324 / 2000 - tt3692652
Item 325 / 2000 - tt0803057
Item 326 / 2000 - tt5629964
Item 327 / 2000 - tt2404425
Item 328 / 2000 - tt3471098
Item 329 / 2000 - tt8998472
Item 330 / 2000 - tt1521197
Item 331 / 2000 - tt0899106
Item 332 / 2000 - tt0970411
Item 333 / 2000 - tt2577172
Item 334 / 2000 - tt

Item 590 / 2000 - tt1049405
Item 591 / 2000 - tt3922798
Item 592 / 2000 - tt1701990
Item 593 / 2000 - tt0288415
Item 594 / 2000 - tt2014400
Item 595 / 2000 - tt1702443
Item 596 / 2000 - tt1014801
Item 597 / 2000 - tt0246989
Item 598 / 2000 - tt0313542
Item 599 / 2000 - tt4978710
Item 600 / 2000 - tt2717860
Item 601 / 2000 - tt0458413
Item 602 / 2000 - tt2396589
Item 603 / 2000 - tt3138376
Item 604 / 2000 - tt0338427
Item 605 / 2000 - tt9326056
Item 606 / 2000 - tt2309961
Item 607 / 2000 - tt4411618
Item 608 / 2000 - tt0404802
Item 609 / 2000 - tt2011311
Item 610 / 2000 - tt4686844
Item 611 / 2000 - tt2167202
Item 612 / 2000 - tt1094198
Item 613 / 2000 - tt1700845
Item 614 / 2000 - tt1178665
Item 615 / 2000 - tt2235902
Item 616 / 2000 - tt0318974
Item 617 / 2000 - tt9577852
Item 618 / 2000 - tt6156350
Item 619 / 2000 - tt1493842
Item 620 / 2000 - tt0414055
Item 621 / 2000 - tt2380331
Item 622 / 2000 - tt0266489
Item 623 / 2000 - tt0436697
Item 624 / 2000 - tt2064968
Item 625 / 2000 - tt

Item 883 / 2000 - tt2281159
Item 884 / 2000 - tt5460858
Item 885 / 2000 - tt0279113
Item 886 / 2000 - tt1648204
Item 887 / 2000 - tt7369722
Item 888 / 2000 - tt2140373
Item 889 / 2000 - tt0466856
Item 890 / 2000 - tt1535612
Item 891 / 2000 - tt1788391
Item 892 / 2000 - tt2954474
Item 893 / 2000 - tt4686108
Item 894 / 2000 - tt1286130
Item 895 / 2000 - tt0268126
Item 896 / 2000 - tt3774790
Item 897 / 2000 - tt1581835
Item 898 / 2000 - tt0159784
Item 899 / 2000 - tt1196956
Item 900 / 2000 - tt0355702
Item 901 / 2000 - tt0246464
Item 902 / 2000 - tt3525168
Item 903 / 2000 - tt5281700
Item 904 / 2000 - tt0780607
Item 905 / 2000 - tt0397044
Item 906 / 2000 - tt2461520
Item 907 / 2000 - tt0261983
Item 908 / 2000 - tt2124787
Item 909 / 2000 - tt2452386
Item 910 / 2000 - tt1381418
Item 911 / 2000 - tt0353324
Item 912 / 2000 - tt0339412
Item 913 / 2000 - tt0323250
Item 914 / 2000 - tt5351000
Item 915 / 2000 - tt3686998
Item 916 / 2000 - tt0304415
Item 917 / 2000 - tt4917224
Item 918 / 2000 - tt

Item 1170 / 2000 - tt2891174
Item 1171 / 2000 - tt4526546
Item 1172 / 2000 - tt2268573
Item 1173 / 2000 - tt1625340
Item 1174 / 2000 - tt0437405
Item 1175 / 2000 - tt6269658
Item 1176 / 2000 - tt1839591
Item 1177 / 2000 - tt3212232
Item 1178 / 2000 - tt4234734
Item 1179 / 2000 - tt0202402
Item 1180 / 2000 - tt0257756
Item 1181 / 2000 - tt2557276
Item 1182 / 2000 - tt4054654
Item 1183 / 2000 - tt0427944
Item 1184 / 2000 - tt1389096
Item 1185 / 2000 - tt4481066
Item 1186 / 2000 - tt3804114
Item 1187 / 2000 - tt2460976
Item 1188 / 2000 - tt0457437
Item 1189 / 2000 - tt1566648
Item 1190 / 2000 - tt2198956
Item 1191 / 2000 - tt0448114
Item 1192 / 2000 - tt2667918
Item 1193 / 2000 - tt7394674
Item 1194 / 2000 - tt5034122
Item 1195 / 2000 - tt1645131
Item 1196 / 2000 - tt0872230
Item 1197 / 2000 - tt0827573
Item 1198 / 2000 - tt0464196
Item 1199 / 2000 - tt1560954
Item 1200 / 2000 - tt2756032
Item 1201 / 2000 - tt1083462
Item 1202 / 2000 - tt5990474
Item 1203 / 2000 - tt8106596
Item 1204 / 20

Item 1451 / 2000 - tt1554092
Item 1452 / 2000 - tt0862467
Item 1453 / 2000 - tt2049543
Item 1454 / 2000 - tt0497116
Item 1455 / 2000 - tt9177040
Item 1456 / 2000 - tt0167456
Item 1457 / 2000 - tt0452592
Item 1458 / 2000 - tt0476958
Item 1459 / 2000 - tt0810945
Item 1460 / 2000 - tt1393000
Item 1461 / 2000 - tt1972571
Item 1462 / 2000 - tt6142496
Item 1463 / 2000 - tt1605717
Item 1464 / 2000 - tt2131532
Item 1465 / 2000 - tt1510984
Item 1466 / 2000 - tt2314886
Item 1467 / 2000 - tt2669336
Item 1468 / 2000 - tt0865907
Item 1469 / 2000 - tt0815244
Item 1470 / 2000 - tt4995790
Item 1471 / 2000 - tt4486986
Item 1472 / 2000 - tt4897822
Item 1473 / 2000 - tt2556874
Item 1474 / 2000 - tt4898730
Item 1475 / 2000 - tt4180032
Item 1476 / 2000 - tt0333766
Item 1477 / 2000 - tt4871980
Item 1478 / 2000 - tt1319704
Item 1479 / 2000 - tt0795368
Item 1480 / 2000 - tt0479760
Item 1481 / 2000 - tt0250323
Item 1482 / 2000 - tt0382189
Item 1483 / 2000 - tt4682136
Item 1484 / 2000 - tt10883004
Item 1485 / 2

Item 1732 / 2000 - tt1781781
Item 1733 / 2000 - tt0316465
Item 1734 / 2000 - tt0997047
Item 1735 / 2000 - tt0180052
Item 1736 / 2000 - tt3715406
Item 1737 / 2000 - tt0804505
Item 1738 / 2000 - tt2039399
Item 1739 / 2000 - tt4383594
Item 1740 / 2000 - tt1777551
Item 1741 / 2000 - tt0118589
Item 1742 / 2000 - tt1790621
Item 1743 / 2000 - tt3919218
Item 1744 / 2000 - tt1734433
Item 1745 / 2000 - tt3824386
Item 1746 / 2000 - tt0462335
Item 1747 / 2000 - tt0423455
Item 1748 / 2000 - tt2083355
Item 1749 / 2000 - tt4972062
Item 1750 / 2000 - tt3335606
Item 1751 / 2000 - tt5304996
Item 1752 / 2000 - tt1507571
Item 1753 / 2000 - tt7857374
Item 1754 / 2000 - tt7983712
Item 1755 / 2000 - tt0421239
Item 1756 / 2000 - tt0470000
Item 1757 / 2000 - tt0160399
Item 1758 / 2000 - tt1340773
Item 1759 / 2000 - tt0283534
Item 1760 / 2000 - tt0318081
Item 1761 / 2000 - tt2622826
Item 1762 / 2000 - tt1486193
Item 1763 / 2000 - tt0431420
Item 1764 / 2000 - tt0805559
Item 1765 / 2000 - tt0409345
Item 1766 / 20

In [31]:
df4 = df[6000:8000]

In [48]:
df4 = webscraper(df4)

Item 1 / 2000 - tt0365885


C:\Users\thren\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 2000 - tt0424136
Item 3 / 2000 - tt0806027
Item 4 / 2000 - tt1603257
Item 5 / 2000 - tt0317042
Item 6 / 2000 - tt1795702
Item 7 / 2000 - tt2361184
Item 8 / 2000 - tt3491962
Item 9 / 2000 - tt2481480
Item 10 / 2000 - tt2395970
Item 11 / 2000 - tt2090629
Item 12 / 2000 - tt1999192
Item 13 / 2000 - tt1698648
Item 14 / 2000 - tt6081670
Item 15 / 2000 - tt1921070
Item 16 / 2000 - tt4661600
Item 17 / 2000 - tt1234654
Item 18 / 2000 - tt1880399
Item 19 / 2000 - tt0486358
Item 20 / 2000 - tt1959438
Item 21 / 2000 - tt0367188
Item 22 / 2000 - tt0282552
Item 23 / 2000 - tt2449638
Item 24 / 2000 - tt4146622
Item 25 / 2000 - tt5566790
Item 26 / 2000 - tt3478232
Item 27 / 2000 - tt0443456
Item 28 / 2000 - tt2781832
Item 29 / 2000 - tt0458438
Item 30 / 2000 - tt0854678
Item 31 / 2000 - tt0415978
Item 32 / 2000 - tt1606392
Item 33 / 2000 - tt0821638
Item 34 / 2000 - tt0184791
Item 35 / 2000 - tt2693580
Item 36 / 2000 - tt5153236
Item 37 / 2000 - tt0342172
Item 38 / 2000 - tt0435528
Item 39 /

Item 297 / 2000 - tt0988849
Item 298 / 2000 - tt0393685
Item 299 / 2000 - tt1422136
Item 300 / 2000 - tt1684226
Item 301 / 2000 - tt2414212
Item 302 / 2000 - tt2183034
Item 303 / 2000 - tt0186045
Item 304 / 2000 - tt0387059
Item 305 / 2000 - tt5352846
Item 306 / 2000 - tt2184287
Item 307 / 2000 - tt1781775
Item 308 / 2000 - tt0415965
Item 309 / 2000 - tt1097013
Item 310 / 2000 - tt0383694
Item 311 / 2000 - tt1229360
Item 312 / 2000 - tt1092633
Item 313 / 2000 - tt0361841
Item 314 / 2000 - tt1654523
Item 315 / 2000 - tt1135095
Item 316 / 2000 - tt1100053
Item 317 / 2000 - tt2977090
Item 318 / 2000 - tt3268340
Item 319 / 2000 - tt1147687
Item 320 / 2000 - tt5536736
Item 321 / 2000 - tt1781058
Item 322 / 2000 - tt0385705
Item 323 / 2000 - tt0804552
Item 324 / 2000 - tt7344360
Item 325 / 2000 - tt5340856
Item 326 / 2000 - tt0493424
Item 327 / 2000 - tt2034031
Item 328 / 2000 - tt3687398
Item 329 / 2000 - tt0787462
Item 330 / 2000 - tt1135933
Item 331 / 2000 - tt0323033
Item 332 / 2000 - tt

Item 590 / 2000 - tt0435665
Item 591 / 2000 - tt4547120
Item 592 / 2000 - tt1867093
Item 593 / 2000 - tt1185431
Item 594 / 2000 - tt2040560
Item 595 / 2000 - tt0258038
Item 596 / 2000 - tt6719524
Item 597 / 2000 - tt2885628
Item 598 / 2000 - tt0179098
Item 599 / 2000 - tt2027154
Item 600 / 2000 - tt0867334
Item 601 / 2000 - tt0439771
Item 602 / 2000 - tt1935156
Item 603 / 2000 - tt6263642
Item 604 / 2000 - tt3505712
Item 605 / 2000 - tt4381236
Item 606 / 2000 - tt3090670
Item 607 / 2000 - tt0800003
Item 608 / 2000 - tt0408196
Item 609 / 2000 - tt0122459
Item 610 / 2000 - tt1714209
Item 611 / 2000 - tt1814643
Item 612 / 2000 - tt0363780
Item 613 / 2000 - tt0454776
Item 614 / 2000 - tt1092634
Item 615 / 2000 - tt1064884
Item 616 / 2000 - tt6649108
Item 617 / 2000 - tt0472062
Item 618 / 2000 - tt0285627
Item 619 / 2000 - tt3910814
Item 620 / 2000 - tt1020543
Item 621 / 2000 - tt1139282
Item 622 / 2000 - tt0772178
Item 623 / 2000 - tt1640711
Item 624 / 2000 - tt0307535
Item 625 / 2000 - tt

Item 883 / 2000 - tt2028530
Item 884 / 2000 - tt0250934
Item 885 / 2000 - tt0818165
Item 886 / 2000 - tt5521550
Item 887 / 2000 - tt2980626
Item 888 / 2000 - tt1407065
Item 889 / 2000 - tt0905994
Item 890 / 2000 - tt0799954
Item 891 / 2000 - tt4872078
Item 892 / 2000 - tt2012665
Item 893 / 2000 - tt0948547
Item 894 / 2000 - tt2334733
Item 895 / 2000 - tt0251806
Item 896 / 2000 - tt2120152
Item 897 / 2000 - tt0491747
Item 898 / 2000 - tt1107859
Item 899 / 2000 - tt5710042
Item 900 / 2000 - tt0448172
Item 901 / 2000 - tt1697851
Item 902 / 2000 - tt1682181
Item 903 / 2000 - tt0314786
Item 904 / 2000 - tt3726704
Item 905 / 2000 - tt1579232
Item 906 / 2000 - tt0204137
Item 907 / 2000 - tt5751998
Item 908 / 2000 - tt1700844
Item 909 / 2000 - tt0271271
Item 910 / 2000 - tt3762912
Item 911 / 2000 - tt0372884
Item 912 / 2000 - tt7584702
Item 913 / 2000 - tt1957945
Item 914 / 2000 - tt3520290
Item 915 / 2000 - tt0413879
Item 916 / 2000 - tt6883152
Item 917 / 2000 - tt4145178
Item 918 / 2000 - tt

Item 1170 / 2000 - tt2499076
Item 1171 / 2000 - tt0803061
Item 1172 / 2000 - tt1545986
Item 1173 / 2000 - tt2112900
Item 1174 / 2000 - tt0358294
Item 1175 / 2000 - tt0488380
Item 1176 / 2000 - tt9876408
Item 1177 / 2000 - tt6582384
Item 1178 / 2000 - tt5790274
Item 1179 / 2000 - tt1186788
Item 1180 / 2000 - tt9095324
Item 1181 / 2000 - tt2566644
Item 1182 / 2000 - tt1276962
Item 1183 / 2000 - tt1217070
Item 1184 / 2000 - tt1982882
Item 1185 / 2000 - tt2100093
Item 1186 / 2000 - tt2386404
Item 1187 / 2000 - tt2392385
Item 1188 / 2000 - tt0395656
Item 1189 / 2000 - tt2355540
Item 1190 / 2000 - tt1838520
Item 1191 / 2000 - tt2066176
Item 1192 / 2000 - tt0961728
Item 1193 / 2000 - tt0349889
Item 1194 / 2000 - tt1320239
Item 1195 / 2000 - tt0425196
Item 1196 / 2000 - tt2415464
Item 1197 / 2000 - tt1564368
Item 1198 / 2000 - tt0302739
Item 1199 / 2000 - tt1757746
Item 1200 / 2000 - tt0842926
Item 1201 / 2000 - tt1523372
Item 1202 / 2000 - tt0432289
Item 1203 / 2000 - tt1715320
Item 1204 / 20

Item 1453 / 2000 - tt7688990
Item 1454 / 2000 - tt6945882
Item 1455 / 2000 - tt4000870
Item 1456 / 2000 - tt0417385
Item 1457 / 2000 - tt6286216
Item 1458 / 2000 - tt2165859
Item 1459 / 2000 - tt5173032
Item 1460 / 2000 - tt4008652
Item 1461 / 2000 - tt0236388
Item 1462 / 2000 - tt1331025
Item 1463 / 2000 - tt1151359
Item 1464 / 2000 - tt1954701
Item 1465 / 2000 - tt0446687
Item 1466 / 2000 - tt1208647
Item 1467 / 2000 - tt3828858
Item 1468 / 2000 - tt2923088
Item 1469 / 2000 - tt1682940
Item 1470 / 2000 - tt1405500
Item 1471 / 2000 - tt1621046
Item 1472 / 2000 - tt1777595
Item 1473 / 2000 - tt1814836
Item 1474 / 2000 - tt1082007
Item 1475 / 2000 - tt0981072
Item 1476 / 2000 - tt5810122
Item 1477 / 2000 - tt2174896
Item 1478 / 2000 - tt2179087
Item 1479 / 2000 - tt1714866
Item 1480 / 2000 - tt1183665
Item 1481 / 2000 - tt4901356
Item 1482 / 2000 - tt1727523
Item 1483 / 2000 - tt0411195
Item 1484 / 2000 - tt7042838
Item 1485 / 2000 - tt1821427
Item 1486 / 2000 - tt2245195
Item 1487 / 20

Item 1735 / 2000 - tt0427038
Item 1736 / 2000 - tt9442842
Item 1737 / 2000 - tt0433350
Item 1738 / 2000 - tt4258698
Item 1739 / 2000 - tt1649780
Item 1740 / 2000 - tt0448090
Item 1741 / 2000 - tt2071491
Item 1742 / 2000 - tt0304328
Item 1743 / 2000 - tt3745906
Item 1744 / 2000 - tt8179218
Item 1745 / 2000 - tt6215044
Item 1746 / 2000 - tt2552394
Item 1747 / 2000 - tt1130992
Item 1748 / 2000 - tt1965162
Item 1749 / 2000 - tt0959306
Item 1750 / 2000 - tt0422779
Item 1751 / 2000 - tt3721954
Item 1752 / 2000 - tt0762110
Item 1753 / 2000 - tt1870527
Item 1754 / 2000 - tt3881680
Item 1755 / 2000 - tt0762111
Item 1756 / 2000 - tt3608654
Item 1757 / 2000 - tt1680051
Item 1758 / 2000 - tt3776288
Item 1759 / 2000 - tt0337881
Item 1760 / 2000 - tt5031014
Item 1761 / 2000 - tt1849868
Item 1762 / 2000 - tt7531138
Item 1763 / 2000 - tt0800226
Item 1764 / 2000 - tt4949290
Item 1765 / 2000 - tt1502420
Item 1766 / 2000 - tt2236160
Item 1767 / 2000 - tt5952266
Item 1768 / 2000 - tt2132486
Item 1769 / 20

In [32]:
df5 = df[8000:10001]

In [52]:
df5 = webscraper(df5)

Item 1 / 2000 - tt0443450


C:\Users\thren\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 2000 - tt3328716
Item 3 / 2000 - tt0247164
Item 4 / 2000 - tt3333870
Item 5 / 2000 - tt1628055
Item 6 / 2000 - tt1552624
Item 7 / 2000 - tt1477855
Item 8 / 2000 - tt2718440
Item 9 / 2000 - tt0940580
Item 10 / 2000 - tt3687186
Item 11 / 2000 - tt1991031
Item 12 / 2000 - tt0156323
Item 13 / 2000 - tt2459156
Item 14 / 2000 - tt2909476
Item 15 / 2000 - tt0462160
Item 16 / 2000 - tt5113926
Item 17 / 2000 - tt4630444
Item 18 / 2000 - tt3194590
Item 19 / 2000 - tt2396224
Item 20 / 2000 - tt1972819
Item 21 / 2000 - tt1198408
Item 22 / 2000 - tt3323638
Item 23 / 2000 - tt3145220
Item 24 / 2000 - tt1312251
Item 25 / 2000 - tt2130142
Item 26 / 2000 - tt2245151
Item 27 / 2000 - tt2378884
Item 28 / 2000 - tt0254334
Item 29 / 2000 - tt4680568
Item 30 / 2000 - tt1447793
Item 31 / 2000 - tt0318763
Item 32 / 2000 - tt0390336
Item 33 / 2000 - tt0430576
Item 34 / 2000 - tt9056818
Item 35 / 2000 - tt1986994
Item 36 / 2000 - tt1563704
Item 37 / 2000 - tt0482461
Item 38 / 2000 - tt3004852
Item 39 /

Item 299 / 2000 - tt2948078
Item 300 / 2000 - tt1957938
Item 301 / 2000 - tt0266391
Item 302 / 2000 - tt0368711
Item 303 / 2000 - tt0289635
Item 304 / 2000 - tt9308456
Item 305 / 2000 - tt0343996
Item 306 / 2000 - tt1638364
Item 307 / 2000 - tt5692030
Item 308 / 2000 - tt2042447
Item 309 / 2000 - tt8106570
Item 310 / 2000 - tt3382842
Item 311 / 2000 - tt5862902
Item 312 / 2000 - tt3704416
Item 313 / 2000 - tt4172224
Item 314 / 2000 - tt0221069
Item 315 / 2000 - tt0430674
Item 316 / 2000 - tt5886510
Item 317 / 2000 - tt0398842
Item 318 / 2000 - tt6864864
Item 319 / 2000 - tt1176724
Item 320 / 2000 - tt0451102
Item 321 / 2000 - tt7825514
Item 322 / 2000 - tt0988083
Item 323 / 2000 - tt0449077
Item 324 / 2000 - tt1855401
Item 325 / 2000 - tt3311384
Item 326 / 2000 - tt2663812
Item 327 / 2000 - tt0271219
Item 328 / 2000 - tt0258273
Item 329 / 2000 - tt5158522
Item 330 / 2000 - tt8271714
Item 331 / 2000 - tt0396857
Item 332 / 2000 - tt0156841
Item 333 / 2000 - tt5456798
Item 334 / 2000 - tt

Item 591 / 2000 - tt0910847
Item 592 / 2000 - tt7410684
Item 593 / 2000 - tt1704292
Item 594 / 2000 - tt1942971
Item 595 / 2000 - tt0851430
Item 596 / 2000 - tt2125653
Item 597 / 2000 - tt1865573
Item 598 / 2000 - tt0239986
Item 599 / 2000 - tt5434972
Item 600 / 2000 - tt1517249
Item 601 / 2000 - tt2689958
Item 602 / 2000 - tt1555126
Item 603 / 2000 - tt1485749
Item 604 / 2000 - tt4711924
Item 605 / 2000 - tt0206367
Item 606 / 2000 - tt0204640
Item 607 / 2000 - tt1865393
Item 608 / 2000 - tt1135924
Item 609 / 2000 - tt1020876
Item 610 / 2000 - tt0366177
Item 611 / 2000 - tt2708254
Item 612 / 2000 - tt6915100
Item 613 / 2000 - tt3205394
Item 614 / 2000 - tt5070196
Item 615 / 2000 - tt0218378
Item 616 / 2000 - tt5700176
Item 617 / 2000 - tt1596753
Item 618 / 2000 - tt10009796
Item 619 / 2000 - tt7545566
Item 620 / 2000 - tt1541123
Item 621 / 2000 - tt6688354
Item 622 / 2000 - tt0944101
Item 623 / 2000 - tt5097070
Item 624 / 2000 - tt1087527
Item 625 / 2000 - tt2475846
Item 626 / 2000 - t

Item 883 / 2000 - tt0360323
Item 884 / 2000 - tt0206917
Item 885 / 2000 - tt3157224
Item 886 / 2000 - tt0182295
Item 887 / 2000 - tt0339034
Item 888 / 2000 - tt3294200
Item 889 / 2000 - tt5278578
Item 890 / 2000 - tt5115546
Item 891 / 2000 - tt1667418
Item 892 / 2000 - tt4441422
Item 893 / 2000 - tt0430919
Item 894 / 2000 - tt3564200
Item 895 / 2000 - tt1533089
Item 896 / 2000 - tt2133239
Item 897 / 2000 - tt1103984
Item 898 / 2000 - tt1368440
Item 899 / 2000 - tt7374952
Item 900 / 2000 - tt3082826
Item 901 / 2000 - tt1785669
Item 902 / 2000 - tt2113659
Item 903 / 2000 - tt3110960
Item 904 / 2000 - tt0381442
Item 905 / 2000 - tt1935065
Item 906 / 2000 - tt5537228
Item 907 / 2000 - tt1772373
Item 908 / 2000 - tt4712076
Item 909 / 2000 - tt7942746
Item 910 / 2000 - tt1844203
Item 911 / 2000 - tt3703148
Item 912 / 2000 - tt3108244
Item 913 / 2000 - tt7647010
Item 914 / 2000 - tt2362991
Item 915 / 2000 - tt3379814
Item 916 / 2000 - tt0421045
Item 917 / 2000 - tt4016250
Item 918 / 2000 - tt

Item 1170 / 2000 - tt9353586
Item 1171 / 2000 - tt2437548
Item 1172 / 2000 - tt3384904
Item 1173 / 2000 - tt3009336
Item 1174 / 2000 - tt1757769
Item 1175 / 2000 - tt0200669
Item 1176 / 2000 - tt3283556
Item 1177 / 2000 - tt0223268
Item 1178 / 2000 - tt0475296
Item 1179 / 2000 - tt0925248
Item 1180 / 2000 - tt0388888
Item 1181 / 2000 - tt6032170
Item 1182 / 2000 - tt4405860
Item 1183 / 2000 - tt0992993
Item 1184 / 2000 - tt0470761
Item 1185 / 2000 - tt3489470
Item 1186 / 2000 - tt1559826
Item 1187 / 2000 - tt6948354
Item 1188 / 2000 - tt1876261
Item 1189 / 2000 - tt2417134
Item 1190 / 2000 - tt1355640
Item 1191 / 2000 - tt2219210
Item 1192 / 2000 - tt0424942
Item 1193 / 2000 - tt2800050
Item 1194 / 2000 - tt4288636
Item 1195 / 2000 - tt0756703
Item 1196 / 2000 - tt2306783
Item 1197 / 2000 - tt0409034
Item 1198 / 2000 - tt0410332
Item 1199 / 2000 - tt2076346
Item 1200 / 2000 - tt5031332
Item 1201 / 2000 - tt4158594
Item 1202 / 2000 - tt0250469
Item 1203 / 2000 - tt4048668
Item 1204 / 20

Item 1453 / 2000 - tt0783532
Item 1454 / 2000 - tt6085974
Item 1455 / 2000 - tt0378407
Item 1456 / 2000 - tt0199727
Item 1457 / 2000 - tt0358500
Item 1458 / 2000 - tt4970632
Item 1459 / 2000 - tt0896585
Item 1460 / 2000 - tt9169592
Item 1461 / 2000 - tt3138104
Item 1462 / 2000 - tt5716380
Item 1463 / 2000 - tt1680140
Item 1464 / 2000 - tt0991150
Item 1465 / 2000 - tt2395123
Item 1466 / 2000 - tt3104930
Item 1467 / 2000 - tt0420901
Item 1468 / 2000 - tt3620452
Item 1469 / 2000 - tt3077108
Item 1470 / 2000 - tt2611390
Item 1471 / 2000 - tt1606390
Item 1472 / 2000 - tt1444252
Item 1473 / 2000 - tt1295072
Item 1474 / 2000 - tt0396688
Item 1475 / 2000 - tt0425308
Item 1476 / 2000 - tt7544820
Item 1477 / 2000 - tt1788785
Item 1478 / 2000 - tt3097084
Item 1479 / 2000 - tt0285441
Item 1480 / 2000 - tt2567038
Item 1481 / 2000 - tt1042916
Item 1482 / 2000 - tt1205071
Item 1483 / 2000 - tt0486259
Item 1484 / 2000 - tt3455224
Item 1485 / 2000 - tt3384870
Item 1486 / 2000 - tt3606848
Item 1487 / 20

Item 1733 / 2000 - tt6794476
Item 1734 / 2000 - tt2798456
Item 1735 / 2000 - tt3503460
Item 1736 / 2000 - tt1647483
Item 1737 / 2000 - tt1340425
Item 1738 / 2000 - tt1029167
Item 1739 / 2000 - tt1683876
Item 1740 / 2000 - tt1286750
Item 1741 / 2000 - tt0460898
Item 1742 / 2000 - tt1378122
Item 1743 / 2000 - tt2249081
Item 1744 / 2000 - tt0800367
Item 1745 / 2000 - tt0496350
Item 1746 / 2000 - tt2396436
Item 1747 / 2000 - tt1757800
Item 1748 / 2000 - tt1378121
Item 1749 / 2000 - tt4126312
Item 1750 / 2000 - tt6540084
Item 1751 / 2000 - tt1667903
Item 1752 / 2000 - tt1421361
Item 1753 / 2000 - tt2246924
Item 1754 / 2000 - tt0340012
Item 1755 / 2000 - tt0338706
Item 1756 / 2000 - tt3809276
Item 1757 / 2000 - tt1130088
Item 1758 / 2000 - tt4906686
Item 1759 / 2000 - tt0219105
Item 1760 / 2000 - tt1560169
Item 1761 / 2000 - tt1173494
Item 1762 / 2000 - tt0289765
Item 1763 / 2000 - tt1254947
Item 1764 / 2000 - tt3317874
Item 1765 / 2000 - tt0779496
Item 1766 / 2000 - tt1720621
Item 1767 / 20

In [74]:
#attach the 5 dataframes together
tmdb_bom = pd.concat([df1, df2, df3, df4, df5])

In [78]:
tmdb_bom

,id,studio,budget,dom_gross,for_gross
0,tt2380307,Walt Disney Studios Motion Pictures,None,210460015,597357873
1,tt7286456,Warner Bros.,55000000,335451311,738800000
2,tt7349950,Warner Bros.,79000000,211593228,261500000
3,tt4520988,Walt Disney Studios Motion Pictures,150000000,477373578,972653355
4,tt7975244,Sony Pictures Entertainment (SPE),125000000,320314960,479744747
...,...,...,...,...,...
9995,tt0349080,None,None,None,None
9996,tt0366178,None,None,None,None
9997,tt2501618,None,None,None,None
9998,tt0340376,Sony Pictures Classics,27000000,765637,314185


In [72]:
# Setting the title as the index
tmdb_bom.set_index('id', inplace=True)

# Data Export

In [73]:
#exporting the dataframe to a csv
tmdb_bom.to_csv('api_data/tmdb_bom_scraped.csv')

# TO DO

Write Docstrings for all functions